In [ ]:
# Function for creating the plots

import json
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text
from collections.abc import Iterable
import os

def load_json(filepath):
    with open(filepath) as file:
        return json.load(file)


def plot_results(results_dirs, experiment, methods, series_result, const_result, seeds, filename, x_axis_label=None, y_axis_label=None, title=None, legend=True):

    fig = plt.figure()
    fig.patch.set_alpha(0)

    plt.clf()
    ax = plt.gca()
    ax.set_facecolor('white')
    max_x = 0
    to_plot = []
    barWidth = 0.1

    j = 0
    for i, (method, label) in enumerate(methods.items()):

        br_base = np.arange(len(results_dirs)) 
        br = [x + j * barWidth for x in br_base] 

        if not isinstance(seeds, Iterable):
            seeds = [seeds]
        
        reg_y_vals = []

        for r_dir in results_dirs:
            y_vals = []

            for seed in seeds:
                try:
                    path = f"{r_dir}/{experiment}/{method}_s={seed}_results.json"
                    if not os.path.exists(path):
                        #print(f"Missing: {path}")
                        y_vals.append(0)
                        continue

                    method_results = load_json(path)
                    if series_result in method_results:
                        y_vals.append(np.array([y for x, y in method_results[series_result]]))
                    else:
                        y_vals.append(method_results[const_result])
                except Exception as e:
                    print(f"Problem with file {path}, error: {e}")
                    #return

            if len(y_vals):
                y_vals = np.array(y_vals) * 100
                y_vals = np.mean(y_vals, axis=0)

                if not isinstance(y_vals, np.ndarray):
                    y_vals = np.array([y_vals])
            
                reg_y_vals.append(y_vals[-1])

        if len(reg_y_vals) == 0:
            continue
        
        j += 1
        plt.bar(br, reg_y_vals, width=barWidth, label=label)
    
    if legend:
        plt.legend(loc='lower right', fontsize=8, ncol=2)

    if title is not None:
        plt.title(title)
    
    # if x_axis_label is not None:
    #     plt.xlabel(x_axis_label)
    
    plt.xticks([r + j / 2 * barWidth for r in range(len(br))], ['0', '1e-6', '1e-3', '0.1', '1'])
    plt.xlabel("$\\lambda$") 

    if y_axis_label is not None:
        plt.ylabel(y_axis_label)

    plt.margins(0.15, 0.25)
    plt.savefig(f"{filename}.pdf", dpi=300, bbox_inches='tight')

In [ ]:
import os
os.makedirs("online_plots_reg_icml_camera_ready", exist_ok=True)

plt.rcParams.update({
    #"figure.figsize": (4, 1.5), # for smaller plots
    "figure.figsize": (3.5, 2),
    #"figure.figsize": (15, 10),
    "figure.dpi": 200,
    "figure.autolayout": False,
    "text.usetex": True,
    'mathtext.fontset': 'stix',
    'text.latex.preamble': "\\usepackage{amsmath}",
    'font.family': 'STIXGeneral',
    'savefig.transparent': False,
})

results_dirs = [
    "../results_online_replicate_new_reg_0",
    "../results_online_replicate_new_reg_1e-6",
    "../results_online_replicate_new_reg_1e-3",
    "../results_online_replicate_new_reg_0.1",
    #"../results_online_replicate_new_reg_0.1_v2",
    "../results_online_replicate_new_reg_1",
    #"../results_online_replicate_old",
    # "../results_online_multiclass_reg_0",
    # "../results_online_multiclass_reg_1e-6",
    # "../results_online_multiclass_reg_1e-3",
    # "../results_online_multiclass_reg_0.1",
    # "../results_online_multiclass_reg_0.1_v2",
    # "../results_online_multiclass_reg_1",
]

seeds = [1, 13, 23, 2024, 7700]
results_dir = "../results_online6"
for dataset, dataset_name in {
        "youtube_deepwalk_plt": "YouTube", 
        "mediamill_plt": "Mediamill", 
        "flicker_deepwalk_plt": "Flickr", 
        # "eurlex_plt": "Eurlex", 
        "eurlex_lexglue_plt": "Eurlex-LexGlue", 
        # "eurlex_lightxml": "Eurlex", 
        "rcv1x_plt": "RCV1-x", 
        "amazoncat_plt": "AmazonCat", 
        #"news20_hsm": "News-20",
        #"ledgar_hsm": "Ledgar-LexGlue",
        # "amazoncat_lightxml": "AmazonCat", 
        # "amazon_lightxml": "Amazon", 
        # "wiki500_lightxml": "Wikipedia",
# }.items():
# seeds = [1, 13, 23]
# results_dir = "../results_online9"
# for dataset, dataset_name in {
#         "youtube_deepwalk_online": "YouTube", 
#         "mediamill_online": "Mediamill", 
#         "flicker_deepwalk_online": "Flickr", 
#         "eurlex_lexglue_online": "Eurlex-LexGlue", 
}.items():
    for k in [0, 3]:
        for measure, short, measure_name in [
                ("micro_f1", f"mF@{k}", "Micro F1"), 
                ("macro_f1", f"mF@{k}", "Macro F1" if k == 0 else f"Macro F1@{k}"), 
                ("macro_recall", f"mR@{k}", "Macro Recall" if k == 0 else f"Macro Recall@{k}"), 
                ("macro_precision", f"mR@{k}", "Macro Precision" if k == 0 else f"Macro Precision@{k}"), 
                ("macro_gmean", f"mG@{k}", "Macro G-mean" if k == 0 else f"Macro G-mean@{k}"),
                ("macro_hmean", f"mH@{k}", "Macro H-Mean"),
                #("multiclass_gmean", f"G@{k}", "Multi-class G-mean"),
                #("multiclass_hmean", f"H@{k}", "Multi-class H-Mean"),
                #("multiclass_qmean", f"Q@{k}", "Multi-class Q-Mean"),
                #("macro_min_tp_tn", f"min_tp_tn@{k}", "Macro-Min(TP,TN)"),
            ]:

            methods = {}

            if measure in ["micro_f1", "macro_f1"] and k == 0:
                methods[f"ofo_{measure}_k={k}"] = "OFO"
            
            if "micro" not in measure:
                methods.update({
                    f"online_greedy_{measure}_k={k}": "Greedy",
                })
                methods.update({
                    f"online_greedy_etu_{measure}_k={k}": "Greedy$(\\hat \\eta)$",
                })

            methods.update({
                f"online_frank_wolfe_exp=1.1_{measure}_k={k}": "Online-FW",
                f"online_frank_wolfe_etu_exp=1.1_{measure}_k={k}": "Online-FW$(\\hat \\eta)$",
                f"omma_{measure}_k={k}": "OMMA",
                f"omma_etu_{measure}_k={k}": "OMMA$(\\hat \\eta)$",
            })

            for series in ["pred_utility_history"]: #["pred_utility_history", "solution_utility_history"]:
                file = f"online_plots_reg_icml_camera_ready/{dataset}_{measure}_k={k}_{series}"
                print(file)
                title = f"{dataset_name} --- {measure_name}"
                if k > 0:
                    title += f"@{k}"
                plot_results(results_dirs, dataset, methods, series, short, seeds, file, 
                             x_axis_label="$\\lambda$", #"Observed instances $(\\boldsymbol{x}, \\boldsymbol{y})$", 
                             y_axis_label=f"$\\text{{{measure_name}}}$ (\%)", 
                             title=None, legend=True)

In [ ]:
import os
os.makedirs("online_plots_reg_icml_camera_ready", exist_ok=True)

plt.rcParams.update({
    #"figure.figsize": (4, 1.5), # for smaller plots
    "figure.figsize": (3.5, 2),
    #"figure.figsize": (15, 10),
    "figure.dpi": 200,
    "figure.autolayout": False,
    "text.usetex": True,
    'mathtext.fontset': 'stix',
    'text.latex.preamble': "\\usepackage{amsmath}",
    'font.family': 'STIXGeneral',
    'savefig.transparent': False,
})

results_dirs = [
    # "../results_online_replicate_new_reg_0",
    # "../results_online_replicate_new_reg_1e-6",
    # "../results_online_replicate_new_reg_1e-3",
    # "../results_online_replicate_new_reg_0.1",
    # #"../results_online_replicate_new_reg_0.1_v2",
    # "../results_online_replicate_new_reg_1",
    # #"../results_online_replicate_old",
    "../results_online_multiclass_reg_0",
    "../results_online_multiclass_reg_1e-6",
    "../results_online_multiclass_reg_1e-3",
    "../results_online_multiclass_reg_0.1",
    #"../results_online_multiclass_reg_0.1_v2",
    "../results_online_multiclass_reg_1",
]

seeds = [1, 13, 23, 2024, 7700]
results_dir = "../results_online6"
for dataset, dataset_name in {
        "news20_hsm": "News-20",
        "ledgar_hsm": "Ledgar-LexGlue",
        "cal256_lr": "Caltech-256",
        # "amazoncat_lightxml": "AmazonCat", 
        # "amazon_lightxml": "Amazon", 
        # "wiki500_lightxml": "Wikipedia",
# }.items():
# seeds = [1, 13, 23]
# results_dir = "../results_online9"
# for dataset, dataset_name in {
#         "youtube_deepwalk_online": "YouTube", 
#         "mediamill_online": "Mediamill", 
#         "flicker_deepwalk_online": "Flickr", 
#         "eurlex_lexglue_online": "Eurlex-LexGlue", 
}.items():
    for k in [1, 3]:
        for measure, short, measure_name in [
                #("micro_f1", f"mF@{k}", "Micro F1"), 
                ("macro_f1", f"mF@{k}", "Macro F1" if k == 0 else f"Macro F1@{k}"), 
                ("macro_recall", f"mR@{k}", "Macro Recall" if k == 0 else f"Macro Recall@{k}"), 
                ("macro_precision", f"mR@{k}", "Macro Precision" if k == 0 else f"Macro Precision@{k}"), 
                #("macro_gmean", f"mG@{k}", "Macro G-mean" if k == 0 else f"Macro G-mean@{k}"),
                #("macro_hmean", f"mH@{k}", "Macro H-Mean"),
                ("multiclass_gmean", f"G@{k}", "Multi-class G-mean"),
                ("multiclass_hmean", f"H@{k}", "Multi-class H-Mean"),
                ("multiclass_qmean", f"Q@{k}", "Multi-class Q-Mean"),
                #("macro_min_tp_tn", f"min_tp_tn@{k}", "Macro-Min(TP,TN)"),
            ]:

            methods = {}
            
            if "micro" not in measure:
                methods.update({
                    f"online_greedy_{measure}_k={k}": "Greedy",
                })
                methods.update({
                    f"online_greedy_etu_{measure}_k={k}": "Greedy$(\\hat \\eta)$",
                })

            methods.update({
                f"online_frank_wolfe_exp=1.1_{measure}_k={k}": "Online-FW",
                f"online_frank_wolfe_etu_exp=1.1_{measure}_k={k}": "Online-FW$(\\hat \\eta)$",
                f"omma_{measure}_k={k}": "OMMA",
                f"omma_etu_{measure}_k={k}": "OMMA$(\\hat \\eta)$",
            })

            for series in ["pred_utility_history"]: #["pred_utility_history", "solution_utility_history"]:
                file = f"online_plots_reg_icml_camera_ready/{dataset}_{measure}_k={k}_{series}"
                print(file)
                title = f"{dataset_name} --- {measure_name}"
                if k > 0:
                    title += f"@{k}"
                plot_results(results_dirs, dataset, methods, series, short, seeds, file, 
                             x_axis_label="$\\lambda$", #"Observed instances $(\\boldsymbol{x}, \\boldsymbol{y})$", 
                             y_axis_label=f"$\\text{{{measure_name}}}$ (\%)", 
                             title=None, legend=True)